In [1]:
import requests
import itertools
from arcgis.geometry import Geometry

import pandas as pd
from arcgis.features import SpatialDataFrame as SDF
from arcgis.features import GeoAccessor, GeoSeriesAccessor, Feature

from arcgis.gis import GIS, Item

import sys
sys.path.append('..')

import geometry_monkeypatch
import reach_tools

In [2]:
reach_item_id = 'c72a6e88a67e4eafb7b291f17ec0cf18'
reach_layer_url = 'https://services5.arcgis.com/12oODIpfxlRR11MF/arcgis/rest/services/Reach_Line/FeatureServer/0'
reach_id_ldub = 2156
reach_id_farmies = 2269
reach_id_canyon = 3066

In [3]:
import importlib
importlib.reload(reach_tools)

reach = reach_tools.Reach.get_from_aw(reach_id_ldub)

reach.update_putin_takeout_and_trace()

reach

In [4]:
gis = GIS(username='joel_mccune')
gis

Enter password: ········


In [5]:
web_map = gis.map()
web_map

MapView(basemap='default', layout=Layout(height='400px', width='100%'))

In [6]:
web_map.extent = reach.geometry.extent  # one of my commits to make life a little easier - ahhhhh!

web_map.draw(
    shape=reach.putin.geometry, 
    symbol={
        "type":"esriPMS",
        "url":"http://static.arcgis.com/images/Symbols/Basic/GreenSphere.png",
        "contentType":"image/png",
        "width":24,
        "height":24
    }
)
web_map.draw(
    shape=reach.takeout.geometry, 
    symbol={
        "type":"esriPMS",
        "url":"http://static.arcgis.com/images/Symbols/Basic/RedSphere.png",
        "contentType":"image/png",
        "width":24,
        "height":24
    }
)
web_map.draw(
    shape=reach.geometry,
    symbol={
        "width": 2.25,
        "color": [0,92,230,255],
        "type": "esriSLS",
        "style": "esriSLSSolid"
      }
)

In [7]:
reach_layer = reach_tools.ReachFeatureLayer(reach_layer_url, gis)

In [8]:
from arcgis.features import FeatureLayer
lyr = FeatureLayer(reach_layer_url, gis)
web_map.add_layer(lyr)
lyr.properties

{
  "defaultVisibility": true,
  "enableZDefaults": true,
  "geometryType": "esriGeometryPolyline",
  "copyrightText": "",
  "hasMetadata": true,
  "supportsCalculate": true,
  "supportsStatistics": true,
  "advancedQueryCapabilities": {
    "supportsTopFeaturesQuery": true,
    "supportsReturningGeometryCentroid": false,
    "supportsMaxRecordCountFactor": true,
    "supportsQueryWithDatumTransformation": true,
    "supportsOrderBy": true,
    "supportsReturningGeometryProperties": true,
    "supportsPaginationOnAggregatedQueries": true,
    "supportsQueryWithResultType": true,
    "supportsQueryRelatedPagination": true,
    "supportsQueryWithDistance": true,
    "supportsStatistics": true,
    "supportsOutFieldSQLExpression": true,
    "supportsPagination": true,
    "supportsReturningQueryExtent": true,
    "supportsDistinct": true,
    "supportsAdvancedQueryRelated": true,
    "supportsSqlExpression": true,
    "supportsHavingClause": true,
    "supportsCountDistinct": true,
    "s

In [9]:
reach_layer.flush()

In [10]:
reach.publish(gis, reach_layer)

{'addResults': [{'globalId': 'DDB1F901-ECCB-49A6-BD0C-BE520214F9A6',
   'objectId': 101,
   'success': True,
   'uniqueId': 101}],
 'deleteResults': [],
 'updateResults': []}

In [135]:
for reach_id in range(0, 10000):
    print('reach_id -', reach_id)
    reach = reach_tools.Reach.get_from_aw(reach_id)
    if type(reach) == reach_tools.Reach:
        trace_status = reach.update_putin_takeout_and_trace()
        if trace_status == True:
            resp = reach.publish(gis, reach_layer)
            print(resp)
        else:
            print('/no trace')
    else:
        print('\tno reach')

reach_id - 0
	no reach
reach_id - 1


TypeError: 'bool' object is not subscriptable

In [140]:
self = reach_tools.Reach.get_from_aw(1)
waters = reach_tools.WATERS()
epa_point = waters.get_epa_snap_point(self.putin.geometry.x, self.putin.geometry.y)
epa_point

False